# 機械学習をPythonで実践する-20　　～ 特徴量選択 ～

In [2]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder, OneHotEncoder
# # import statsmodels.api as sma
from sklearn.model_selection import train_test_split ,cross_val_score, KFold, RepeatedKFold,StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
from sklearn.compose import ColumnTransformer
# from sklearn import tree
# from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,AdaBoostClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from lightGBM_cv import lightGBM_classifier_cv_func
from category_encoders import TargetEncoder

%matplotlib inline
import matplotlib.pyplot as plt


## Greedy Feature Selection

In [14]:
dtypes = {
    "species": str,
    'island': str,
    'culmen_length_mm': pl.Float32, # くちばしの長さ[mm]
    'culmen_depth_mm': pl.Float32, # くちばしの高さ[mm]
    'flipper_length_mm': pl.Float32, # 翼の長さ[mm]
    'body_mass_g': pl.Float32, # 体重[g]
    'sex': str
}

# ペンギンのデータセット読み込み。欠損値がNAとして含まれているので、null_values="NA"を指定しないと読み込みエラーになる。
df = pl.read_csv('../Python/sample_data/ML_sample/penguins_size.csv',dtypes=dtypes, null_values='NA')


### - 前処理

In [15]:
df.null_count()

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
u32,u32,u32,u32,u32,u32,u32
0,0,2,2,2,2,10


In [16]:
# sexカラムの.は欠損値扱いとする。
df = df.with_columns(
    (pl.when(pl.col('sex') == '.').then(None).otherwise(pl.col('sex'))).alias('sex')
)

In [17]:
# 欠損値が多すぎる行（値が入っている列が3つより少ない行）を削除する。
# この操作はPolarsだと面倒なので、一回Pandasに変換してやる。
df = pl.from_pandas(df.to_pandas().dropna(thresh=3))

In [18]:
df.null_count()

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,9


In [19]:
target = 'species'
X = df.drop(target)
y = df.get_column(target)

In [20]:
# sexカラムの欠損値を文字列の'NaN'で置き換える
X = X.fill_null('NaN')
X.null_count()

island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


今回は欠損値を新たなカテゴリ'NaN'として扱うだけなので、欠損値対応を交差検証の中で行う必要はない。  
この段階でOne-hot Encodingによるダミー変数化を行うと、greedy feature selectionによる特徴量選択がダミー変数も含む状態で行われ、  
計算量が多くなる、かつ元の特徴量を部分的に使うことになってしまいわかりづらくなってしまうので、  
ダミー変数化はpipelineにしてgreedy feature selectionのCVの中で行う。

### - 特徴量エンジニアリング

ここでは多項式特徴量と四則演算した結果を用いる。

In [12]:
# 多項式特徴量
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_length_depth = poly.fit_transform(X.select(['culmen_length_mm', 'culmen_depth_mm']))

In [21]:
X = X.with_columns([
    pl.Series(poly_length_depth[:, 0]).alias('culmen_length_mm'),
    pl.Series(poly_length_depth[:, 1]).alias('culmen_depth_mm'),
    pl.Series(poly_length_depth[:, 2]).alias('culmen_length_mm^2'),
    pl.Series(poly_length_depth[:, 3]).alias('culmen_length_X_depth'),
    pl.Series(poly_length_depth[:, 4]).alias('culmen_depth_mm^2')
])

In [24]:
# culmen_length_mm, culmen_depth_mmの差と比を計算する
X = X.with_columns([
    (pl.col('culmen_length_mm') - pl.col('culmen_depth_mm')).alias('culmen_diff'),
    (pl.col('culmen_length_mm') / pl.col('culmen_depth_mm')).alias('culmen_ratio'),
])

In [26]:
X.head(3)

island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex,culmen_length_mm^2,culmen_length_X_depth,culmen_depth_mm^2,culmen_diff,culmen_ratio
str,f64,f64,f32,f32,str,f64,f64,f64,f64,f64
"""Torgersen""",39.099998,18.700001,181.0,3750.0,"""MALE""",1528.809881,731.170001,349.690029,20.399998,2.090909
"""Torgersen""",39.5,17.4,186.0,3800.0,"""FEMALE""",1560.25,687.299985,302.759987,22.1,2.270115
"""Torgersen""",40.299999,18.0,195.0,3250.0,"""FEMALE""",1624.089939,725.399986,324.0,22.299999,2.238889


In [30]:
X.dtypes

[Utf8,
 Float64,
 Float64,
 Float32,
 Float32,
 Utf8,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64]

### - CVの準備

In [36]:
# カテゴリカラムと数値カラムのリストを取得
num_cols = X.select(pl.col(pl.Float64)).columns
cat_cols = X.select(pl.col(pl.Utf8)).columns

In [39]:
# 欠損値対応後にダミー変数に変換すべきなので、ここでやってはダメ。
#X = pd.get_dummies(X.to_pandas(), drop_first=False)

# cvインスタンスを生成。3 fold で評価する。
cv = KFold(n_splits=3, random_state=0, shuffle=True)

# 多クラス分類の場合、LightGBMのデータセットに入れる目的変数はラベルエンコーディングしておく必要がある。
encoder = LabelEncoder()
y_encoded = pd.DataFrame(encoder.fit_transform(y.to_pandas()), columns=[target])

# LightGBM用のパラメタ指定
params = {
          'objective': 'multiclass',  # 最小化させるべき損失関数
          'num_class': 3,  # マルチクラス分類の場合、クラス数を指定する必要あり。
          'metric': 'multi_logloss',  # 学習時に使用する評価指標(early_stoppingの評価指標にも同じ値が使用される)
          'learning_rate': 0.01,
          'max_depth': 5,
          'random_state': 0,  # 乱数シード
          'boosting_type': 'goss',  # boosting_type
          'verbose': -1  # これを指定しないと`No further splits with positive gain, best gain: -inf`というWarningが表示される
         }

In [41]:
# 前処理用のColumnTransformerを定義
preprocess_ct = ColumnTransformer([('std_scale', OneHotEncoder(drop='first',handle_unknown='ignore', sparse_output=False), num_cols),
                                   ('dummies', StandardScaler(), cat_cols)])

### - Greedy feature selectionクラスの定義

In [ ]:
class GreedyFeatureSelection():
    def __init__(self) -> None:
        